## Markov Chain(MC):

A Markov chain is the mathematical model for stochastic processes where the future state dependes solely on the present state and not on the sequence of events that preceded it. this property is known as the markov property . MC used in various fields such as genetics, economics, game theory, and computer science.

### Key Concepts:

1. **States** : The different possible conditions or positions in which the system can exist.
2. **Transition Probability** : The probability of moving from one state to another.
3. **Transition Matrix** : A matrix that represents the transition probabilities between states.
4. **Initial State Distribution** : The probabilities of starting in each state.

### Example:

Consider a simple weather model with two states: "Sunny"
and "Rainy". The transition probabilities are as follows:

- If today is Sunny, there is a 0.8 probability that tomorrow will be Sunny and a 0.2 probability that it will be Rainy.
- If today is Rainy, there is a 0.4 probability that tomorrow will be Sunny and a 0.6 probability that it will be Rainy.
  The transition matrix for this Markov chain can be represented as:

```
        | Sunny | Rainy |
-------------------------
Sunny  |  0.8  |  0.2  |
Rainy  |  0.4  |  0.6  |
```

The initial state distribution could be:

```
        | Sunny | Rainy |
-------------------------
        |  0.5  |  0.5  |
```

This indicates that there is a 50% chance of starting in either state.

### Applications:

- **Weather Forecasting** : Predicting future weather conditions based on current data.
- **Stock Market Analysis** : Modeling stock price movements.
- **Natural Language Processing** : Text generation and speech recognition.
- **Game Theory** : Analyzing strategies in games with probabilistic outcomes.
  Markov chains provide a powerful framework for modeling systems that evolve over time with inherent randomness.


## Mathematical representation:

Imagine a system with a finite set of states, say $ S = \{s_1, s_2, \dots, s_n\} $. At each time step $ t $, the system is in one state $ X_t \in S $.

Transition probabilities: The probability of moving from state $ i $ to state $ j $ is $ P*{ij} = P(X*{t+1} = s_j \mid X_t = s_i) $. These are fixed and don't change with time (assuming a time-homogeneous chain).

All $ P*{ij} \geq 0 $ and for each $ i $, $ \sum_j P*{ij} = 1 $ (rows sum to 1).

We represent this as a transition matrix $ P $, an $ n \times n $ matrix where entry (i,j) is $ P\_{ij} $.

Initial state distribution: The initial probabilities of being in each state can be represented as a vector $ \pi^{(0)} = [\pi_1^{(0)}, \pi_2^{(0)}, \dots, \pi_n^{(0)}] $, where $ \pi_i^{(0)} = P(X_0 = s_i) $.

The state distribution at time $ t $ can be computed as:
$$ \pi^{(t)} = \pi^{(0)} P^t $$
where $ P^t $ is the matrix $ P $ raised to the power of $ t $.


In [ ]:
import random
def simulate_markov_chain(transition_matrix,intial_state,num_steps):
    curr_state=intial_state
    state_history=[curr_state]

    for _ in range(num_steps):
        probs=transition_matrix[curr_state]
        next_state=random.choices(range(len(probs)),weights=probs)[0]
        curr_state=next_state
        state_history.append(curr_state)
    return state_history


transition_matrix=[[0.9,0.1],[0.5,0.5]]

initial_state=0
num_steps=10

print(simulate_markov_chain(transition_matrix,initial_state,num_steps))


[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]


In [6]:
def estimate_steady_state(transition_matrix,initial_state,num_steps,num_trials=1000):
    state_counts=[0]*len(transition_matrix)
    for _ in range(num_trials):
        states=simulate_markov_chain(transition_matrix,initial_state,num_steps)
        state_counts[states[-1]]+=1
    steady_state=[count/num_trials for count in state_counts]
    return steady_state

print(estimate_steady_state(transition_matrix,initial_state,num_steps))


[0.843, 0.157]


In [3]:
# Input:
#   N = number of states
#   transition = N x N matrix (list of lists), transition[i][j] = P(go from i to j)
#   start = initial state (0 to N-1)
#   T = number of steps

# Output:
#   List of T+1 states (including start)

# Sample:
#   N=2
#   transition = [[0.8, 0.2], [0.3, 0.7]]
#   start = 0
#   T = 3
# Output: [0, 0, 1, 0] (example; random)

In [8]:
import random

def simulate_path(transition,start,T):
    N=len(transition)
    path=[start]
    current=start

    for _ in range(T):
        probs=transition[current]
        next_state=random.choices(range(N),weights=probs)[0]
        current=next_state
        path.append(current)
    return path


transition_matrix=[[0.8,0.2],[0.3,0.7]]
simulate_path(transition_matrix,0,3)

[0, 0, 0, 1]

In [ ]:
# Input:
#   P = transition matrix (list of lists)
#   N = steps
#   i, j = start and target states

# Output:
#   float = P^N[i][j]

# Sample:
#   P = [[0.8, 0.2], [0.3, 0.7]]
#   N=2, i=0, j=1
# Output: 0.26

In [13]:
def matrix_multiply(A,B):
    n=len(A)
    C=[[0.0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j]+=A[i][k]*B[k][j]
    return C


def matrix_power(P,N):
    result=[[1 if i==j else 0 for j in range(len(P))] for i in range(len(P))]
    while N>0:
        if N%2==1:
            result=matrix_multiply(result,P)
        P=matrix_multiply(P,P)
        N//=2
    return result

def prob_n_steps(P,N,i,j):
    P_N=matrix_power(P,N)
    return P_N[i][j]

P = [[0.8, 0.2], [0.3, 0.7]]
print(prob_n_steps(P, 2, 0, 1))

0.30000000000000004


In [14]:
# Above code has three loop means O(n^3) time complexity

import numpy as np

def matrix_power_np(P,N):
    n=P.shape[0]
    result=np.eye(n)

    while N>0:
        if N%2==1:
            result=result@P
        P=P@P
        N//=2
    return result


def prob_n_step_np(P,N,i,j):
    P=np.array(P,dtype=float)
    P_N=matrix_power_np(P,N)
    return P_N[i][j]


P = np.array([[0.8, 0.2],
              [0.3, 0.7]])

print(prob_n_step_np(P, 2, 0, 1))

0.30000000000000004


In [22]:
import numpy as np
import time

def matrix_multiply(A, B):
    n = len(A)
    C = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    return C

def matrix_power_py(P, N):
    result = [[1 if i == j else 0 for j in range(len(P))] for i in range(len(P))]
    while N > 0:
        if N % 2 == 1:
            result = matrix_multiply(result, P)
        P = matrix_multiply(P, P)
        N //= 2
    return result

def matrix_power_np(P, N):
    result = np.eye(P.shape[0])
    while N > 0:
        if N % 2 == 1:
            result = result @ P
        P = P @ P
        N //= 2
    return result

def benchmark(matrix_size=10, power=50):
    P = np.random.rand(matrix_size, matrix_size)
    P /= P.sum(axis=1, keepdims=True)
    P_list = P.tolist()

    print(f"\nMatrix: {matrix_size}x{matrix_size}, Power: {power}")

    start_py = time.perf_counter()
    _ = matrix_power_py(P_list, power)
    end_py = time.perf_counter()
    print(f"Pure Python Time: {end_py - start_py:.6f} s")

    start_np = time.perf_counter()
    _ = matrix_power_np(P.copy(), power)
    end_np = time.perf_counter()
    print(f"NumPy Time:       {end_np - start_np:.6f} s")

    print(f"Speedup: {(end_py - start_py) / (end_np - start_np):.2f}x")

benchmark(matrix_size=10, power=50)
benchmark(matrix_size=50, power=50)
benchmark(matrix_size=100, power=50)



Matrix: 10x10, Power: 50
Pure Python Time: 0.000665 s
NumPy Time:       0.000043 s
Speedup: 15.42x

Matrix: 50x50, Power: 50
Pure Python Time: 0.067783 s
NumPy Time:       0.000149 s
Speedup: 454.00x

Matrix: 100x100, Power: 50
Pure Python Time: 0.539847 s
NumPy Time:       0.001002 s
Speedup: 538.99x
